# Advanced Numpy & Pandas

[Python Module of the Week](https://github.com/INM-6/Python-Module-of-the-Week) by [Alex van Meegen](https://alexvanmeegen.github.io/)

## Table of contents
1. [Numpy](#numpy)
    1. [Anatomy of an ndarray](#anatomy)
    2. [Views & Copies](#viewscopies)
    3. [Broadcasting](#broadcasting)
3. [Pandas](#pandas)
    1. [Series & DataFrames](#seriesdataframes)
    2. [Multiindex](#multiindex)
    3. [Groupby](#groupby)

## Numpy <a name="numpy"></a>

In [ ]:
import numpy as np

In [ ]:
def print_info(a):
    print('number of elements:', a.size)
    print('number of dimensions:', a.ndim)
    print('shape:', a.shape)
    print('data type:', a.dtype)
    print('strides:', a.strides)

### Anatomy of an ndarray <a name="anatomy"></a>

* [ndarray](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) = block of memory + indexing scheme + data type descriptor

<img src="img/array.png" alt="array" style="width: 600px;"/>

In [ ]:
a = np.arange(6)
print(a)

In [ ]:
print_info(a)

* `dtype` changes the size of the elements -> strides change:

In [ ]:
a = np.arange(6, dtype=np.int16)
print_info(a)

In [ ]:
a = np.arange(6, dtype=np.float32)
print_info(a)

* now 2d arrays:

<img src="img/strides.png" alt="strides" style="width: 700px;"/>

In [ ]:
a = np.arange(9).reshape(3, 3)
print(a)

In [ ]:
print_info(a)

### Views & Copies <a name="viewscopies"></a>

* [view](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.view.html): looking from different angles at the underlying block of memory
* [copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.copy.html): assigns a new block of memory

In [ ]:
print_info(a)
print('-'*25)
print_info(a.T)

* `a.T` returns a view, with immediate consequences:

In [ ]:
print(a)
a.T[2, 2] = -1
print(a)
a[2, 2] = 8
print(a)

* beyond transposition:

In [ ]:
print(a.ravel())
print('-'*25)
print_info(a)
print('-'*25)
print_info(a.ravel())

* how to identify views?

In [ ]:
print(a.ravel())
print(a.ravel().base)
print(a.ravel().base is a.base)

In [ ]:
print(a.flatten())
print(a.flatten().base)
print(a.flatten().base is a.base)

* [indexing](https://docs.scipy.org/doc/numpy/user/basics.indexing.html): returns a view
* [fancy indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing): returns a copy

In [ ]:
print(a)
print(a.base)

In [ ]:
print(a[0:2, :])
print(a[0:2, :].base)

In [ ]:
print(a[[0, 2], :])
print(a[[0, 2], :].base)

In [ ]:
print(a[a > 3])
print(a[a > 3].base)

* avoiding copies: in-place operations

In [ ]:
print(a)
np.add(a, 5, out=a)
print(a)
np.subtract(a, 5, out=a)
print(a)

* of course, we can manipulate the `dtype`, the `strides`, ...
* this can lead to some weird stuff (but might also be handy sometimes)

In [ ]:
print(a)
print_info(a)
print('-'*25)
print(a.view(dtype=np.int16))
print_info(a.view(dtype=np.int16))

### Broadcasting <a name="broadcasting"></a>

* [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html): making array shapes compatible, e.g. in when multiplying with a scalar
* manually: use `np.newaxis` to create dummy axis
* internal: adds a 0 length stride

In [ ]:
a = np.arange(6)
print(a)

In [ ]:
b = a[:, np.newaxis]
print(b)
print(b.base is a)
print_info(b)

* why would we want to do this?
* to compute the outer product $a_i a_j$

In [ ]:
print(a * b)

* or to compute the distances between points $d_{ij}^2 = (\vec{x}_j - \vec{x}_i)^2 = \sum_{k=1}^d (x_{jk} -x_{ik})^2$

In [ ]:
points = np.array([[1, 0, 2], [-2, 0, 3], [2, 1, 2], [0, 0, 1]])
print(points)
print(points.shape)
square_dist = np.sum((points[np.newaxis, :, :] - points[:, np.newaxis, :])**2, axis=2)
print(square_dist)

* let's do a fancy example: calculating a correlation function
$$ C_{\phi}(\tau) = \lim_{t\to\infty} \langle \phi[x(t)] \phi[x(t+\tau)] \rangle, \quad x \sim \mathrm{GP}(0, C_{x}) $$
* using Gauss-Hermite quadrature, the integral can be approximated by
$$ C_{\phi}(\tau) \approx \frac{1}{2\pi}\sum_{j=1}^{n}w_{j}\sum_{i=1}^{n}w_{i}\phi[\beta(\tau)x_{j}-\alpha(\tau)x_{i}]\phi[\beta(\tau)x_{j}+\alpha(\tau)x_{i}] $$

In [ ]:
from scipy.special import roots_hermitenorm

def corrfct_gauss_hermite(C_x, phi, n=20):
    alpha = np.sqrt(0.5 * (C_x[0] - C_x))[:, np.newaxis, np.newaxis]
    beta = np.sqrt(0.5 * (C_x[0] + C_x))[:, np.newaxis, np.newaxis]

    x, w = roots_hermitenorm(n) 
    x_i = x[np.newaxis, :, np.newaxis]
    x_j = x[np.newaxis, np.newaxis, :]
    w_i = w[np.newaxis, :, np.newaxis]
    w_j = w[np.newaxis, np.newaxis, :]

    return np.sum(w_j * w_i * phi(beta*x_j-alpha*x_i) * phi(beta*x_j+alpha*x_i), axis=(1, 2)) / (2*np.pi)

In [ ]:
import matplotlib.pyplot as plt

t = 0.01*np.arange(1001)
C_x = np.cos(np.pi*t)/np.cosh(t/2)

fig, axs = plt.subplots(1, 2, figsize=(16, 4))
for ax in axs:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_xlim(t.min(), t.max())
    ax.set_xlabel(r'$\tau$')
    ax.set_ylabel(r'$C_{\phi}(\tau)$')
axs[0].plot(t, corrfct_gauss_hermite(C_x, np.tanh), 'o', c='steelblue')
axs[0].set_title(r'$\phi(x)=\tanh(x)$')
axs[1].plot(t, corrfct_gauss_hermite(C_x, np.exp), 'o', c='lightsteelblue')
axs[1].plot(t, np.exp(C_x[0]+C_x), 'k-')
axs[1].set_title(r'$\phi(x)=\exp(x)$')
plt.tight_layout()
plt.show()

print(f'maximum relative error: {np.abs(corrfct_gauss_hermite(C_x, np.exp)/np.exp(C_x[0]+C_x)-1).max()}')

* a note on *code vectorization* and *problem vectorization*
* we want to compute $\sum_{i,j} x_i y_j$:

In [ ]:
x = np.random.choice(range(1000), 100)
y = np.random.choice(range(1000), 100)

In [ ]:
def outer_sum_1(x, y):
    return np.sum(x[:, np.newaxis] * y[np.newaxis, :], axis=(0, 1))

In [ ]:
%timeit outer_sum_1(x, y)

* however, we know $\sum_{i,j} x_i y_j = (\sum_i x_i)(\sum_j y_j)$ so let's use this:

In [ ]:
def outer_sum_2(x, y):
    return np.sum(x) * np.sum(y)

In [ ]:
print(np.allclose(outer_sum_1(x, y), outer_sum_2(x, y)))

In [ ]:
%timeit outer_sum_2(x, y)

### Final remarks
* numpy is awesome because it feels very often like writing (vector) equations
* if you `conda install` numpy, it is uses [MKL optimizations](https://docs.anaconda.com/mkl-optimizations/)
  * MKL: Intel Math Kernel Library, library of optimized math routines for Intel processors
  * i.e. once the Python overhead becomes insignificat (large matrices) you probably can't be faster
  * not just basic linear algebra but also things like FFT
* many more *awesome* numpy examples in Nicolas Rougier's book [From Python to Numpy](https://www.labri.fr/perso/nrougier/from-python-to-numpy/)
* for more technical details look at the [Scipy Lecture Notes](http://scipy-lectures.org/advanced/advanced_numpy/index.html)

## Pandas <a name="pandas"></a>

In [ ]:
import pandas as pd

## Series & DataFrames <a name="seriesdataframes"></a>

* [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html): annotated 1d numpy array steroids
* [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html): annotated 2d numpy array on steroids

In [ ]:
C_x = pd.Series(data=C_x, index=t, name='C_x')
print(C_x)

In [ ]:
points = pd.DataFrame(data=points, columns=('x', 'y', 'z'), index=('A', 'B', 'C', 'D'))
print(points)

* how to get the index / columns:

In [ ]:
print(points.index)
print(points.columns)
print(C_x.index)

* getting back the underlying numpy array

In [ ]:
print(points.to_numpy())

* accessing the elements using the labels: `loc`

In [ ]:
print(points.loc[['A', 'C'], :])

In [ ]:
print(points.loc[:, 'x'])

* accessing the elements using the position (like numpy): `iloc`

In [ ]:
print(points.iloc[0:2, :])

* boolean indexing

In [ ]:
print(C_x[C_x > 0.999])

* `pandas` is fully compatible with `numpy`, `scipy`, `matplotlib`, ...

In [ ]:
print(np.sin(points))

In [ ]:
plt.plot(C_x)

* but `numpy` might destroy the precious index structure:

In [ ]:
def rotation_z(theta):
    R = np.zeros((3, 3), dtype=np.float)
    R[0, 0] = R[1, 1] = np.cos(2*np.pi*theta/360)
    R[0, 1] = -np.sin(2*np.pi*theta/360)
    R[1, 0] = np.sin(2*np.pi*theta/360)
    R[2, 2] = 1
    return R

print(np.einsum('ab,ib->ia', rotation_z(45), points))

* neat: index is respected when performing operations

In [ ]:
scaling = pd.Series(data=[0.5, 1, 1.5, 2], index=('B', 'A', 'C', 'D'))
print(scaling)
print(points)
print(points.mul(scaling, axis=0))

* a bit of convenience functions

In [ ]:
print(points.mean())
print(points.std())

In [ ]:
points.apply(np.cumsum, axis=1)

## Multiindex <a name="multiindex"></a>
* [hierarchical indices](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html): makes it possible to represent higher dimensional data

In [ ]:
points_unstacked = points.T.unstack()
print(points_unstacked)

* getting things from the multiindex

In [ ]:
points_unstacked.loc[('A', ['x', 'z'])]

In [ ]:
points_unstacked.loc[(slice(None), 'x')]

* let's do more more realistic example

In [ ]:
neurons = pd.read_pickle('data/neurons.pkl')
print(neurons)

In [ ]:
synapses = pd.read_pickle('data/synapses.pkl')
print(synapses.info())

* total number of excitatory neurons:

In [ ]:
print(neurons.loc[(slice(None), slice(None), 'E')].sum())

* number of synapses between excitatory neurons in 'bankssts' and 'middletemporal'

In [ ]:
print(synapses.loc[('middletemporal', slice(None), slice(None)), 
                   ('bankssts', slice(None), 'E')])

## Groupby <a name="groupby"></a>
* aggregate and apply operation

In [ ]:
print(synapses.groupby(level='layer').sum())

* for a nice example tutorial, see [Modern Pandas](http://tomaugspurger.github.io/modern-1-intro.html)